In [1]:
import cv2
import numpy as np
import os

In [2]:
def show_images(imgList):
    while True:
        for (name,img) in imgList:
            cv2.imshow(name,img)
        pressed_key = cv2.waitKey(0)
        if pressed_key==27:
            break
    cv2.destroyAllWindows()

In [3]:
def jacob_warp_affine(x,y):
    return np.array([[x,0,y,0,1,0],[0,x,0,y,0,1]],dtype=np.float32) ##(2,3)

In [4]:
def linearize_warp(wf):
    assert (wf.shape==(2,3))
    return np.float32([[wf[0,0]-1.0],[wf[1,0]],[wf[0,1]],[wf[1,1]-1.0],[wf[0,2]],[wf[1,2]]]) ##(6,1)

In [5]:
def make_warp(p):
    assert (p.shape==(6,1))
    return np.float32([[p[0,0]+1.0,p[2,0],p[4,0]],[p[1,0],p[3,0]+1.0,p[5,0]]]) ##(2,3)

In [6]:
def inverse_param(p):
    assert (p.shape==(6,1))
    denom = ((1+p[0,0])*(1+p[3,0]) - (p[1,0]*p[2,0]))
    invp = np.zeros(p.shape,dtype=np.float32)
    invp[0,0] = (-p[0,0])-(p[0,0]*p[3,0])+(p[1,0]*p[2,0])
    invp[1,0] = (-p[1,0])
    invp[2,0] = (-p[2,0])
    invp[3,0] = (-p[3,0])-(p[0,0]*p[3,0])+(p[1,0]*p[2,0])
    invp[4,0] = (-p[4,0])-(p[3,0]*p[4,0])+(p[2,0]*p[5,0])
    invp[5,0] = (-p[5,0])-(p[0,0]*p[5,0])+(p[1,0]*p[4,0])
    invp = invp/denom
    return invp ##(6,1)

In [7]:
def compose_param(p1,p2):
    assert (p1.shape==(6,1))
    assert (p2.shape==(6,1))
    comp = np.zeros(p1.shape,dtype=np.float32)
    comp[0,0] = p1[0,0]+p2[0,0]+(p1[0,0]*p2[0,0])+(p1[2,0]*p2[1,0])
    comp[1,0] = p1[1,0]+p2[1,0]+(p1[1,0]*p2[0,0])+(p1[3,0]*p2[1,0])
    comp[2,0] = p1[2,0]+p2[2,0]+(p1[0,0]*p2[2,0])+(p1[2,0]*p2[3,0])
    comp[3,0] = p1[3,0]+p2[3,0]+(p1[1,0]*p2[2,0])+(p1[3,0]*p2[3,0])
    comp[4,0] = p1[4,0]+p2[4,0]+(p1[0,0]*p2[4,0])+(p1[2,0]*p2[5,0])
    comp[5,0] = p1[5,0]+p2[5,0]+(p1[1,0]*p2[4,0])+(p1[3,0]*p2[5,0])
    return comp ##(6,1)

In [8]:
def gaussNewton(img,template,warp,epsilon,transform):
    template = template.astype('float32')
    img = img.astype('float32')
    assert (len(img.shape)==2), "Grayscale image should be given"
    ## template is the target image
    ## img is initial image
    ## warp is the initial guess warp ([1+p1,p3,p5],[p2,1+p4,p6])
    ## transform is one of "AFFINE" "TRANSLATIONAL"
    ## epsilon defines the ending condition
    

    template_shape = template.shape
    tmplx,tmply = template_shape
    
    ## Calculating gradients (3)
    
    # kerX = np.float32([[-0.5,0.0,0.5]])
    # kerY = np.float32([-0.5,0.0,0.5])
    # gradX = cv2.filter2D(template,-1,kerX)
    # gradY = cv2.filter2D(template,-1,kerY)
    
    gradTx = cv2.Sobel(template,cv2.CV_64F,1,0,ksize=3) ##(x,y)
    gradTy = cv2.Sobel(template,cv2.CV_64F,0,1,ksize=3) ##(x,y)
    grad = np.stack([gradTx,gradTy],axis=-1) ##(x,y,2)
    grad = np.expand_dims(grad,axis=2) ##(x,y,1,2)
    
    ## Calculating jacobian of template image (4)
    jacob_tmpl = np.zeros([tmplx,tmply,2,6]) ## (x,y,2,6)
    for i in range(tmplx):
        for j in range(tmply):
            # because i,j in numpy image correspond to j,i in image axis
            jacob_tmpl[i,j] = jacob_warp_affine(j,i)
    
    ## Calculating steepest descent (5)
    steep_desc = np.matmul(grad,jacob_tmpl) ##(x,y,1,6)
    steep_desc_trans = np.transpose(steep_desc,[0,1,3,2]) ##(x,y,6,1)
    
    
    ## Calculating Hessian matrix (6)
    hess = np.sum(np.sum(np.matmul(steep_desc_trans,steep_desc),axis=0),axis=0) ##(6,6)
    inv_hess = np.linalg.inv(hess) ##(6,6)
    
    delP = np.ones([6,1],dtype=np.float32)
    iterations = 0
#     while(np.linalg.norm(delP,2) > epsilon): #2-Norm end condition
    while(np.linalg.norm(delP) > epsilon): #Frobenius norm end condition
        
        ## Calculation warp of given image with current guess (1)
        warp_img = cv2.warpAffine(img,warp,(tmply,tmplx)) ##(x,y)
        ## Calculate error image (2)
        err_img = warp_img - template ##(x,y)
        err_img = np.expand_dims(np.expand_dims(err_img,-1),-1) ##(x,y,1,1)
        
        ## Computer other term (7)
        other_term = np.sum(np.sum(np.matmul(steep_desc_trans,err_img),axis=0),axis=0) ##(6,1)
        ## Computing delP (8)
        delP = np.matmul(inv_hess,other_term) ##(6,1)
        
        ## Updating warp (9)
        initP = linearize_warp(warp) ##(6,1)
#         invP = inverse_param(delP) ##(6,1)
        invP = delP
        nextP = compose_param(initP,invP) ##(6,1)
        warp = make_warp(nextP) ##(2,3)
        iterations += 1
        
#         print("Iteration %d ; Norm %f" %(iterations,np.linalg.norm(delP,ord='fro')))
        
        if iterations > 5000:
            break
    return warp
    

In [9]:
def lukasKanadeTracker(img1,img2,initialwarp,epsilon, transform='AFFINE'):
    img1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
    return gaussNewton(img1,img2,initialwarp,epsilon,transform)

## Example

In [10]:
# img1 = cv2.imread("./images/image1.jpg")
# img2 = cv2.imread("./images/image2.jpg")

In [ ]:
img1 = cv2.imread("./images/room_left.jpg")
img2 = cv2.imread("./images/room_right.jpg")

In [ ]:
# img = cv2.imread("./images/room_left.jpg")
# img1 = img[50:350,50:450]
# img2 = img[55:355,55:455]

In [ ]:
show_images([("img1",img1),("img2",img2)])

In [ ]:
r = cv2.selectROI("Select window of interest",img=img1,fromCenter=False,showCrossair=False)
cv2.destroyAllWindows()
tmpl = img1[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
initial_warp = np.float32([[1.0,0.0,-float(r[0])],[0.0,1.0,-float(r[1])]])

final_warp = lukasKanadeTracker( img2, tmpl, initial_warp, 0.0010, transform='AFFINE')
final_warp[0,2] -= initial_warp[0,2]
final_warp[1,2] -= initial_warp[1,2]

In [ ]:
initial_warp = np.float32([[1.0,0.0,0],[0.0,1.0,0]])
final_warp = lukasKanadeTracker( img2, img1, initial_warp, 0.0010, transform='AFFINE')

In [ ]:
warped_img = cv2.warpAffine(img2,final_warp,(img1.shape[1],img1.shape[0]))

In [ ]:
show_images([("img1",img1),("img2",img2),("warp",warped_img)])

In [ ]:
warp2 = warped_img.copy()
warp3 = warped_img.copy()

In [ ]:
warp2[warp2==0] = img1[warp2==0]
warp3[warp3==0] = img2[warp3==0]

In [ ]:
cv2.selectROI("Select window of interest",img=img1,fromCenter=False,showCrossair=False)

In [ ]:
sho

In [ ]:
cv2.destroyAllWindows()

In [ ]:
show_images([("wp1",warp2),("wp2",warp3)])

## Test video

In [18]:
video_file = "./videos/shaky_book.mov"
result_path = "./results/video/shaky_book/"
os.makedirs(result_path,exist_ok=True)
os.makedirs(result_path+"stable",exist_ok=True)
os.makedirs(result_path+"unstable",exist_ok=True)

In [19]:
cols,rows = (400,600)

In [25]:
cap = cv2.VideoCapture(video_file)
ret,frame=  cap.read()
resz_frame = cv2.resize(frame,(rows,cols))
r = cv2.selectROI("Select window of interest",img=resz_frame,fromCenter=False,showCrossair=False)
cv2.destroyAllWindows()
tmpl = resz_frame[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
initial_warp = np.float32([[1.0,0.0,-float(r[0])],[0.0,1.0,-float(r[1])]])
warp = initial_warp.copy()

In [26]:
stable_frames = []
unstable_frames = []

In [27]:
show_images([("tmpl",tmpl)])

In [28]:
cnt = 0
while True:
    ret,frame=  cap.read()
    if not ret:
        break
    resz_frame = cv2.resize(frame,(rows,cols))
    unstable_frames.append(resz_frame)
    print(cnt)
    print("Starting")
    warp = lukasKanadeTracker( resz_frame, tmpl, warp, 0.0010, transform='AFFINE')
    frame_warp = warp.copy()
    frame_warp[0,2] -= initial_warp[0,2]
    frame_warp[1,2] -= initial_warp[1,2]
    warped_img = cv2.warpAffine(resz_frame,frame_warp,(rows,cols))
    stable_frames.append(warped_img)
    print("Done")
    cnt+=1
#     if cnt>5000:
#         break
#     cv2.imshow("video",resz_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()

0
Starting
Done
1
Starting
Done
2
Starting
Done
3
Starting
Done
4
Starting
Done
5
Starting
Done
6
Starting
Done
7
Starting
Done
8
Starting
Done
9
Starting
Done
10
Starting
Done
11
Starting
Done
12
Starting
Done
13
Starting
Done
14
Starting
Done
15
Starting
Done
16
Starting
Done
17
Starting
Done
18
Starting
Done
19
Starting
Done
20
Starting
Done
21
Starting
Done
22
Starting
Done
23
Starting
Done
24
Starting
Done
25
Starting


KeyboardInterrupt: 

In [ ]:
for i in range(len(stable_frames)):
    cv2.imshow("stable",stable_frames[i])
    path = result_path+"stable/"+('{:04}'.format(i))+".jpg"
    print(path)
    cv2.imwrite(path,stable_frames[i])
    cv2.imshow("unstable",unstable_frames[i])
    cv2.imwrite(result_path+"unstable/"+('{:04}'.format(i))+".jpg",unstable_frames[i])
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()

In [ ]:
img = unstable_frames[144]

In [ ]:
show_images([("im1",tmpl),("im3",unstable_frames[144])])

In [ ]:
warped_img = cv2.warpAffine(img,warp,(tmpl.shape[1],tmpl.shape[0]))

In [ ]:
show_images([("im1",tmpl),("im3",img)])

In [ ]:
warp = lukasKanadeTracker( img, tmpl, initial_warp, 0.0010, transform='AFFINE')

In [ ]:
warp

In [1]:
import numpy as np

In [6]:
x = np.zeros([400,600,3])
y = np.zeros([400,600,3])
divider = np.zeros([x.shape[0],20,x.shape[2]],dtype=x.dtype)

In [7]:
x.shape

(400, 600, 3)

In [8]:
divider.shape

(400, 20, 3)

In [11]:
ans = np.concatenate((x,divider,y),axis=1)

In [12]:
ans.shape

(400, 1220, 3)

In [13]:
import os

In [17]:
os.path.split("./repotr/hah.mp4")[0]

('./repotr', 'hah.mp4')